In [ ]:
import numpy as np
from scipy.optimize import minimize_scalar
from sklearn.datasets import make_blobs
import time

El presente trabajo práctico aborda el problema de Fermat-Weber, que consiste en encontrar la ubicación óptima de un único punto que minimice la suma de las distancias ponderadas a un conjunto de ubicaciones dadas en el plano. Cada una de estas ubicaciones tiene un peso asociado, que representa su importancia relativa dentro del problema.

Este tipo de modelo tiene múltiples aplicaciones prácticas en problemas de localización, logística y planificación, y en este caso se enmarca en la decisión de instalar un centro de atención médica temporal en un área afectada.

El objetivo del trabajo es comparar distintas estrategias de resolución para este problema, analizando su desempeño frente a diferentes configuraciones de puntos y pesos. Para ello, se implementaron tres métodos de optimización:

* Algoritmo de Weiszfeld (original y modificado),


* Descenso coordenado, y

* Dirección de descenso con subgradientes.

Se probaron estos métodos sobre varias instancias con características variadas, evaluando su tiempo de ejecución, estabilidad y número de iteraciones hasta converger. A continuación, se detallan las implementaciones y resultados obtenidos.



## Algoritmo de Weiszfeld

In [ ]:
def weiszfeld(p, w, x0, tol=1e-5, iter_max=1000):
  start = time.time()
  x_actual = x0

  iteraciones = 0
  while iteraciones < iter_max:
    distancias = []
    for i in range(len(p)):
      distancia = np.linalg.norm(x_actual - p[i])
      if distancia < 1e-12: # x = p
        x_sig = p[i]
        return x_sig, iteraciones, time.time() - start
      distancias.append(np.linalg.norm(x_actual - p[i]))

    numerador = sum((w[i] * p[i]) / distancias[i] for i in range(len(p)))
    denominador = sum(w[i] / distancias[i] for i in range(len(p)))
    x_sig = numerador / denominador

    if np.linalg.norm(x_sig - x_actual) < tol:
      return x_sig, iteraciones, time.time() - start

    x_actual = x_sig
    iteraciones += 1

  return x_sig, iteraciones, time.time() - start

In [ ]:
def weiszfeld_modificado(p, w, x0, tol=1e-6, max_iter=1000, eps=1e-12):
    p = np.asarray(p, dtype=float)
    m, _ = p.shape

    x = x0 # centroide ponderado
    start = time.perf_counter()

    for k in range(max_iter):
        dist = np.linalg.norm(p - x, axis=1)

        # verifico singularidad, ie, si x es igual a algun p_j
        j = np.argmin(dist)
        if dist[j] < eps:
            inds_i = np.arange(m) != j # indices de los puntos distintos de j
            diff_j = p[inds_i] - p[j]           # pi - pj
            dist_j = np.linalg.norm(diff_j, axis=1)

            # R_j = suma_{i≠j} w_i (p_j - p_i)/||p_j - p_i||
            R = (w[inds_i][:, None] * (-diff_j / dist_j[:, None])).sum(axis=0)
            norm_R = np.linalg.norm(R)

            if norm_R <= w[j]:  # optimo alcanzado
                return p[j], k, time.perf_counter() - start

            # paso correctivo S(p_j)
            d_j = -R / norm_R                 # direccion de descenso normalizada
            denom = (w[inds_i] / dist_j).sum()  # suma__{i≠j} w_i / ||pi - pj||
            t_j = (norm_R - w[j]) / denom     # longitud del paso
            x_next = p[j] + d_j * t_j
        else:
            # paso estandar de Weiszfeld
            inv_dist = w / dist
            x_next = (inv_dist[:, None] * p).sum(axis=0) / inv_dist.sum()

        if np.linalg.norm(x_next - x) < tol:
            return x_next, k + 1, time.perf_counter() - start

        x = x_next

    return x, max_iter, time.perf_counter() - start

El algoritmo de Weiszfeld busca el **punto fijo** de la función  
$
T(x)\;=\;\frac{\sum_{i=1}^n \tfrac{w_i\,p_i}{\|x - p_i\|}}{\sum_{i=1}^n \tfrac{w_i}{\|x - p_i\|}}
$

partiendo de una estimación inicial $x^{(0)}\notin P$ (en nuestro caso el centroide ponderado).

Buscamos este punto fijo porque T(x) es equivalente a las condiciones de optimalidad $\nabla f(x)=0\$ de nuestro problema con

$
f(x)=\sum_{i=1}^n w_i \,\|x - p_i\|.
$

Un punto fijo $x^{*}$ cumple esta condición, lo que nos garantiza que sea un punto estacionario, o bien un minimo global (por ser f convexa).

En cada iteración k, calculamos  
$
x^{(k+1)}
\;=\;
\frac{\displaystyle\sum_{i=1}^n \frac{w_i\,p_i}{\big\|x^{(k)} - p_i\big\|}}
     {\displaystyle\sum_{i=1}^n \frac{w_i}{\big\|x^{(k)} - p_i\big\|}}
$

hasta que $\|x^{(k+1)} - x^{(k)}\|$ sea menor que alguna tolerancia (1-e5 en nuestro caso).
De esta manera cada $x^{(k+1)}$ es un **promedio ponderado inverso** de las posiciones $p_i$, donde los pesos dinámicos son $w_i / \|x^{(k)}-p_i\|$.

Este algoritmo es un metodo **ad hoc** para este problema ya que se deriva directamente de las condiciones de optimalidad del problema de minimzar $\sum_{i=1}^n w_i\,\|x - p_i\|$ y no aplicamos un esquema generico de optimización (como por ej. el gradiente), sino que reponderamos cada distancia $\|x-p_i\|$ usando los terminos $
\frac{w_i}{\|x - p_i\|}
$ que aparecen al diferenciar la suma de distancias.  

Decidimos utilizar la modificación 1 del algoritmo de Weiszfeld (presentada en clase).

En esta modificación, verificamos singularidad (la iteración coincide con un punto $p_j$) al inicio y calculamos el vector

$
R \;=\;\sum_{i\neq j} w_i\,\frac{p_j - p_i}{\|p_j - p_i\|}
$

y luego si $\|R\|>w_j$, realizamos un paso correctivo

$
x^{(k+1)}
= S(p_j)
= p_j \;-\; \frac{R}{\|R\|}\,
  \frac{\|R\| - w_j}{\sum_{i\neq j} \tfrac{w_i}{\|p_j - p_i\|}}.
$

Caso contrario, utilizamos el paso estandar de Weiszfeld.

Esta modificacion evita la indefinicion y las oscilaciones cerca de los puntos $p_j$, nos garantiza una disminucion monotona de la funcion objetivo aun en caso de singularidad y al hacer un desplazamiento optimo en el entorno de un punto $p_j$, mejoramos la velocidad de convergencia.

## Método de descenso coordenado


In [ ]:
def f(x, puntos, pesos):
    total = 0.0
    for i in range(len(puntos)):
        p = puntos[i]         # punto i
        w = pesos[i]          # peso i
        distancia = np.linalg.norm(x - p)  # distancia euclídea ||x - p_i||
        total += w * distancia            # suma ponderada
    return total

def descenso_coordenado(puntos, pesos, x0, tol=1e-5, max_iter=1000):
    x = np.array(x0, dtype=float)
    n = len(x)                     # Cantidad de coordenadas
    iter = 0
    start = time.time()
    for k in range(max_iter):
        k += 1
        x_anterior = x.copy()     # Guardamos x actual para comparar

        # Recorremos cada coordenada (0 para x, 1 para y)
        for i in range(n):
            def f_univar(lambd):
                x_temp = x.copy()
                x_temp[i] += lambd  # Solo cambiamos la coordenada i
                return f(x_temp, puntos, pesos)

            resultado = minimize_scalar(f_univar)  # Minimiza en λ ∈ ℝ
            paso = resultado.x
            x[i] += paso  # Actualiza la coordenada i

        # Criterio de parada: si el cambio es pequeño, terminamos
        if np.linalg.norm(x - x_anterior) < tol:
            break

    return x, k, time.time() - start


En el código, la función f calcula la suma ponderada de distancias euclídeas entre un punto candidato 𝑥 y un conjunto de puntos dados. Para cada punto, se mide su distancia a 𝑥, se multiplica por su peso y se acumulan estos valores para obtener el resultado total.

Para resolver el problema se utiliza el método de descenso coordenado, un algoritmo iterativo que actualiza una coordenada a la vez. En cada iteración, se minimiza la función variando únicamente la coordenada seleccionada mientras las demás se mantienen fijas, recorriendo las coordenadas de forma **cíclica**. Para ello, se define una función univariante que depende solo de la coordenada actual, y se emplea la función minimize_scalar de SciPy para encontrar el valor óptimo en esa dirección. Posteriormente, se actualiza la coordenada con este valor y se continúa con la siguiente.

El **criterio de parada** se basa en la magnitud del cambio entre la solución actual y la anterior: cuando la norma de la diferencia entre ambos puntos es menor que una tolerancia preestablecida, se considera que el algoritmo ha convergido y se detiene. En caso contrario, el proceso continúa hasta alcanzar un máximo de iteraciones definidas.

Este método es adecuado para funciones convexas como la que plantea el problema de Fermat-Weber, y presenta la **ventaja** de no requerir el cálculo explícito de derivadas, lo cual es relevante ya que la función no es diferenciable en los puntos donde la solución coincide con alguno de los puntos dados.



## Algoritmo de Dirección de Descenso (utilizando el subgradiente de la función)

In [ ]:
#IMPLEMENTACIÓN COMO EN EL CODIGO
#SE UTILIZÓ EL MÉTODO DE SUBGRADIENTE COMO PASO DE DECISIÓN
def f(x, puntos, pesos):
    total = 0.0
    for i in range(len(puntos)):
        p = puntos[i]         # punto i
        w = pesos[i]          # peso i
        distancia = np.linalg.norm(x - p)  # distancia euclídea ||x - p_i||
        total += w * distancia            # suma ponderada
    return total

def direccion_de_descenso(p, w,x0,tol=1e-5, max_iter=1000):
    start = time.time()
    k = 0
    c1 = 0.5
    x_k = x0.copy()
    while (k < max_iter):
        subgradiente_k = np.zeros_like(x0)
        for i in range(len(p)):
            distancia = np.linalg.norm(x_k - p[i])
            if distancia > 0:
                subgradiente_k += w[i] * ((x_k - p[i])/distancia)
        if np.linalg.norm(subgradiente_k) < tol:
            break
        f_k = f(x_k,p,w)
        a_k = 1

        while (f(x_k - a_k*subgradiente_k, p ,w) > f_k - c1*a_k*np.dot(subgradiente_k, subgradiente_k)):
            a_k = a_k * 0.5

        x_k = x_k - a_k * subgradiente_k
        k = k+1
    return x_k, k, time.time() - start

Para este algoritmo se utilizó la misma función f que para el algoritmo de descenso coordinado. Decidimos utilizar el subgradiente de la función como paso de decisión, ya que en ciertos casos el gradiente se podía indefinir. Esto ocurre en los casos (x_k = pi).

El código del algoritmo sigue el pseudocódigo genérico para estos algoritmos. Este método es iterativo; busca minimizar la función f, y en cada iteración vuelve a calcular y utilizar el subgradiente, ya que este muestra una dirección en la que f no aumenta. Una vez tomada esta, se actualizan los valores y sigue iterando. Se utilizó el criterio de armijo (en el último while), que busca asegurar que en cada iteración se disminuya lo suficiente la función.

Mientras no se cumpla el criterio de parada o se llegue a la cantidad máxima de iteraciones fijadas el programa continua iterando. A propósito de esto, el criterio de parada para este algoritmo es que el subgradiente llegue a un valor menor al prefijado (llamado tol en el código).

Este algoritmo en general logra minimizar f en una buena medida. Además tiene la ventaja de que funciona con funciones no diferenciables. Sin embargo, su velocidad de convergencia no es una virtud; aún más, dadas ciertas instancias puede llegar a requerir muchas iteraciones y consecuentemente tardar más que otros algoritmos. Es un método útil cuando la función no es diferenciable (ya que resuelve el problema del gradiente indefinido), pero si esta sí fuera diferenciable, en general conviene utilizar como paso de decisión el gradiente simplemente.

## Instancias de prueba


Para evaluar el desempeño de los algoritmos implementados se diseñaron seis instancias de prueba con características diversas. Estas instancias buscan representar distintos escenarios que podrían surgir en aplicaciones reales del problema de Fermat-Weber, incluyendo situaciones con clústeres, outliers, pesos desbalanceados y espacios de alta dimensión.

La **primera instancia** presenta tres clústeres bien separados entre sí, lo que obliga al algoritmo a encontrar un punto de compromiso que minimice la distancia ponderada a todos ellos. Se espera que la solución se ubique cerca del centro ponderado entre los grupos.

La **segunda instancia** distribuye puntos de forma aproximadamente uniforme en el plano, buscando simular una cobertura amplia y homogénea del área afectada. En este caso, la solución óptima debería coincidir con el centro geométrico de la distribución de los puntos.

La **tercera instancia** introduce un único punto con un peso muy elevado, en comparación con otros puntos livianos distribuidos aleatoriamente. Este escenario simula la presencia de una ubicación con altísima prioridad médica, y se espera que el centro óptimo se sitúe muy próximo a ese punto, ya que es "el mas importante".

La **cuarta instancia** combina un grupo de puntos relativamente agrupados con un outlier lejano, pero con el mismo peso. Este caso permite observar cómo los métodos responden ante valores extremos: la solución no debería quedar ni en el centroide del grupo ni en el outlier, sino en una posición intermedia.

La **quinta instancia** contiene puntos alineados en una línea horizontal con diferentes pesos, lo cual permite evaluar el comportamiento de los métodos cuando la solución debe desplazarse a lo largo de una única dirección influenciada por los pesos asignados.

Finalmente, la **sexta instancia** busca probar qué tan bien funcionan los métodos cuando se trabaja con muchos datos y en un espacio más complejo. Para eso, se generaron 20.000 puntos al azar en un espacio de cinco dimensiones. Este tipo de prueba sirve para ver si los algoritmos son eficientes y rápidos también cuando el problema se vuelve más grande y difícil de visualizar.

🔹 Instancia 1: Múltiples clústeres separados

Tres grupos bien definidos en el plano. Se espera que el centro esté en una posición de compromiso entre los clústeres.



In [ ]:
instancias = []

In [ ]:
def clusters_separados(n_samples, centers, std, box):
    X, y = make_blobs(
        n_samples=n_samples,
        centers=centers,
        cluster_std=std,
        center_box=box,
        random_state=42
    )
    return X

pts_clusters = clusters_separados(16000, centers=16, std=5.0, box=(-4000,4000))
pesos_clusters = np.random.rand(16000)
instancia_clusters = pts_clusters, pesos_clusters
instancias.append(instancia_clusters)

🔹 Instancia 2: Puntos bien dispersos.

La solución debería estar en el centro geométrico.



In [ ]:
def dispersos_uniforme(n_samples, xlim, ylim):
    xs = np.random.uniform(xlim[0], xlim[1], size=n_samples)
    ys = np.random.uniform(ylim[0], ylim[1], size=n_samples)
    return np.column_stack((xs, ys))

pts_dispersos = dispersos_uniforme(16000, (-4000,4000), (-4000,4000))
pesos_dispersos = np.random.rand(16000)
instancia_dispersos = pts_dispersos, pesos_dispersos
instancias.append(instancia_dispersos)

🔹 Instancia 3: Un punto muy pesado entre puntos livianos



In [ ]:
def punto_pesado(base_pts, base_weights, heavy_weight):
    pts = base_pts.copy()
    w = base_weights.copy()
    center = np.mean(base_pts, axis=0)
    pts = np.vstack([pts, center])
    w = np.concatenate([w, [heavy_weight]])
    return pts, w

# 19999 puntos liviano + 1 con peso = 20000
w_base = np.ones(19999)
pts_base = dispersos_uniforme(19999, (0,100), (0,100))
pts_pesados, weights_pesados = punto_pesado(pts_base, w_base, heavy_weight=1000)
instancia_pesados = pts_pesados, weights_pesados
instancias.append(instancia_pesados)

🔹 Instancia 4: Un outlier lejano con el mismo peso

Cuatro puntos agrupados cerca del origen y un outlier. Se espera que la solución se desplace hacia el outlier, aunque no totalmente.


In [ ]:
def outlier_lejano(base_pts, base_weights, outlier_coord):
    pts = np.vstack([base_pts, outlier_coord])
    w = np.concatenate([base_weights, [base_weights.mean()]])
    return pts, w

pts_outlier, weights_outlier = outlier_lejano(pts_dispersos, np.ones(len(pts_dispersos)), (10000,10000))
instancia_outlier = pts_outlier, weights_outlier
instancias.append(instancia_outlier)


🔹 Instancia 5: Puntos alineados con pesos distintos

Todos los puntos están en la misma línea horizontal. La solución debería estar más cerca del punto que tiene peso mayor.

In [ ]:
def alineados_pesos(n_samples, xlim):
    xs = np.linspace(xlim[0], xlim[1], n_samples)
    pts = np.column_stack((xs, np.zeros(n_samples)))
    w = np.random.randint(1, 10, size=n_samples)
    return pts, w

pts_linea, weights_linea = alineados_pesos(20000, (0,10000))
instancia_linea = pts_linea, weights_linea
instancias.append(instancia_linea)

🔹 Instancia 6: Puntos random en $R^{5}$

In [ ]:
n = 20000
puntos_6 = np.random.rand(n, 5)
pesos_6 = np.random.rand(n)
instancia_6 = puntos_6, pesos_6
instancias.append(instancia_6)

## Evaluacion de las instancias:

In [ ]:
for i in range(6):
  print(f"##### Instancia {i+1} #####\n")
  points = instancias[i][0]
  weights = instancias[i][1]
  x_inicial = sum(weights[i]*points[i] for i in range(len(points))) / sum(weights[i] for i in range(len(points))) # centroide ponderado
  sol_w, iter_w, time_w = weiszfeld(points, weights, x_inicial)
  print("WEISZFELD:", sol_w, f"en {time_w:.4f}s", f"({iter_w} iteraciones)")
  sol_wm, iter_wm, time_wm = weiszfeld_modificado(points, weights, x_inicial)
  print("WEISZFELD MODIFICADO:", sol_wm, f"en {time_wm:.4f}s", f"({iter_wm} iteraciones)")
  sol_c, iter_c, time_c = descenso_coordenado(points, weights, x_inicial)
  print("DESCENSO COORDENADO:", sol_c, f"en {time_c:.4f}s", f"({iter_c} iteraciones)")
  sol_d, iter_d, time_d = direccion_de_descenso(points, weights, x_inicial, max_iter=100)
  print("DIRECCION DE DESCENSO ", sol_d, f"en {time_d:.4f}s", f"({iter_d} iteraciones)\n")

##### Instancia 1 #####

WEISZFELD: [-983.15508966 -564.8287985 ] en 6.5003s (31 iteraciones)
WEISZFELD MODIFICADO: [-983.15508633 -564.82880676] en 0.0382s (36 iteraciones)
DESCENSO COORDENADO: [-983.15513055 -564.82865791] en 19.5100s (5 iteraciones)
DIRECCION DE DESCENSO  [-983.15504816 -564.82890131] en 218.0915s (100 iteraciones)

##### Instancia 2 #####

WEISZFELD: [-5.28104312  5.20232842] en 4.7101s (20 iteraciones)
WEISZFELD MODIFICADO: [-5.28104754  5.20232553] en 0.0252s (24 iteraciones)
DESCENSO COORDENADO: [-5.28101126  5.20237055] en 13.6725s (3 iteraciones)
DIRECCION DE DESCENSO  [-5.28102269  5.20234334] en 252.0964s (100 iteraciones)

##### Instancia 3 #####

WEISZFELD: [50.03814008 50.1564903 ] en 0.1303s (0 iteraciones)
WEISZFELD MODIFICADO: [50.03814008 50.1564903 ] en 0.0030s (0 iteraciones)
DESCENSO COORDENADO: [50.03814008 50.1564903 ] en 4.8612s (1 iteraciones)
DIRECCION DE DESCENSO  [50.03814008 50.1564903 ] en 531.2828s (100 iteraciones)

##### Instancia 4 ###

Los resultados para las seis instancias se muestran en la siguiente tabla. En cada caso se indica cuántas iteraciones necesitó cada método y cuánto tardó en ejecutarse. Como todos los métodos llegaron a soluciones prácticamente iguales, no se incluye el valor exacto de la función objetivo.


| Instancia | Método               | Iteraciones | Tiempo (s) |
| --------- | -------------------- | ----------- | ---------- |
| **1**     | Weiszfeld            | 31          | 6.5003     |
|           | Weiszfeld modificado | 36          | 0.0382     |
|           | Desc. coordenado     | 5           | 19.5100    |
|           | Dir. descenso        | 100 (máx)   | 218.0915   |
| **2**     | Weiszfeld            | 20          | 4.7101     |
|           | Weiszfeld modificado | 24          | 0.0252     |
|           | Desc. coordenado     | 3           | 13.6725    |
|           | Dir. descenso        | 100 (máx)   | 252.0964   |
| **3**     | Weiszfeld            | 0           | 0.1303     |
|           | Weiszfeld modificado | 0           | 0.0030     |
|           | Desc. coordenado     | 1           | 4.8612     |
|           | Dir. descenso        | 100 (máx)   | 531.2828   |
| **4**     | Weiszfeld            | 20          | 3.5084     |
|           | Weiszfeld modificado | 24          | 0.0259     |
|           | Desc. coordenado     | 4           | 20.3151    |
|           | Dir. descenso        | 100 (máx)   | 247.9203   |
| **5**     | Weiszfeld            | 69          | 17.9941    |
|           | Weiszfeld modificado | 70          | 0.0899     |
|           | Desc. coordenado     | 2           | 11.4947    |
|           | Dir. descenso        | 100 (máx)   | 395.8716   |
| **6**     | Weiszfeld            | 3           | 0.8483     |
|           | Weiszfeld modificado | 6           | 0.0132     |
|           | Desc. coordenado     | 2           | 20.8391    |
|           | Dir. descenso        | 100 (máx)   | 383.9023   |


Aunque todos llegan a soluciones similares, no todos lo hacen con la misma eficiencia ni estabilidad.

El **algoritmo de Weiszfeld (original)** fue bastante rápido en la mayoría de los casos, pero mostró algunas limitaciones. En particular, cuando la solución coincide exactamente con uno de los puntos (como en la instancia 3), el algoritmo se detiene enseguida y puede no dar la mejor respuesta si no está bien tratado ese caso.

La versión **modificada de Weiszfeld**, en cambio, resultó ser la más confiable y rápida en todos los experimentos. Siempre encontró la solución en pocos pasos y en tiempos muy bajos, incluso en instancias grandes o en espacios más complejos (como la instancia 6 en cinco dimensiones). Además, resolvió correctamente casos donde el algoritmo original podía fallar. Por eso, es el método que más conviene usar en la práctica.

Es importante observar el resultado de la instancia 5 (todos los puntos alineados), donde obtuvimos un mejor tiempo en el algoritmo de descenso coordenado que en el weiszfeld sin modificar. Esto se debe a que Weiszfeld recalcula todas las distancias y necesita formar la media reponderada (calcular x_k+1), mientras que el metodo de descenso coordenado solo necesita operar sobre la unica coordenada relevante, logrando una convergencia mas rapida debido a las pocas evaluaciones unidimensionales que debe hacer

El **método de descenso coordenado** también funcionó bien. Aunque tarda un poco más que Weiszfeld modificado, siempre llega a la solución con pocas iteraciones. Su mayor tiempo se debe a que en cada paso hace una búsqueda unidimensional bastante precisa. Si bien no es el más rápido, es bastante estable y consistente, y puede ser una buena alternativa si se quiere evitar usar derivadas.
.

Por otro lado, el **método de dirección de descenso** (basado en subgradientes) fue el que más problemas presentó. En todas las instancias llegó al máximo de iteraciones permitidas sin cumplir el criterio de parada, lo que muestra que no logró converger del todo. Además, sus tiempos fueron muchísimo más altos que los del resto (en algunos casos tardó varios minutos). Aunque teóricamente válido, no es una buena opción práctica para este tipo de problema, sobre todo cuando los datos son grandes o hay muchas dimensiones.



En **conclusión**, los resultados muestran que cada método tiene ventajas y limitaciones según el caso. El Weiszfeld modificado fue el más rápido y estable, siendo claramente el más adecuado para aplicar en la práctica. El descenso coordenado también cumplió bien, aunque tarda un poco más, pero es una opción sólida. Por otro lado, el método de dirección de descenso presentó varias dificultades y no resultó confiable en muchos casos, por lo que no se recomienda para este problema en su forma actual. En definitiva, elegir el método correcto depende de las características específicas del problema y de los recursos disponibles.